In [1]:
#Cargar la extensión autoreload
%load_ext autoreload
%autoreload 2
# Retriever tools
from libs.chatbot import Chatbot
from libs.vector_retriever import VectorRetriever
from libs.lexical_retriever import LexicalRetriever
from libs.basic_document_storage import BasicStorage
from libs.hybrid_retriever import HybridRetriever
from libs.pdf_loader import PDFLoader
from libs.basic_text_splitter import BasicTextSplitter

# Prompt Tools
from prompts.academic_assistant_prompt import system_prompt as academic_system_prompt
from prompts.academic_assistant_prompt import user_prompt as academic_user_prompt

In [2]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
# Load the documents
pdf_loader = PDFLoader(file_filter={".pdf"})
documents = pdf_loader.load_data()
# Split the documents in chunks
splitter = BasicTextSplitter()
chunked_documents = splitter.split_documents(documents)
# Store the documents
basic_storage = BasicStorage()
basic_storage.add_documents(chunked_documents)


In [3]:
# 2. Create a retriever to handle the retrieval process.
# For handling the storage and retrieval of data chunks with semantic search
semantic_retriever = VectorRetriever()
# For handling the storage and retrieval of data chunks with lexical search
lexical_retriever = LexicalRetriever()
# A retriever for handling all the retrieval process
hybrid_retriever = HybridRetriever(storage_= basic_storage, retrievers=[semantic_retriever,lexical_retriever])
# Add the documents to the retriever
print("Adding documents to the retriever")
hybrid_retriever.add(chunked_documents)


Adding documents to the retriever


WARNING clustering 659 points to 25 centroids: please provide at least 975 training points


BM25S Create Vocab:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/659 [00:00<?, ?it/s]

In [4]:
bot = Chatbot(hybrid_retriever,
              system_prompt=academic_system_prompt,
              user_prompt=academic_user_prompt
              )

In [5]:
query1 = "Cuales son las distintas formas de turismo"
query2= "Hablame de los pasos principales de la organización y el desarrollo de un viaje y los tipos de entidades participantes en ello"

In [6]:
r1 =bot.reply(query1)

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

context='El contexto proporcionado incluye una lista de 17 formas de turismo diferentes, entre las que se encuentran el turismo de sol y playa, turismo cultural, turismo de reuniones, turismo deportivo, turismo náutico, turismo académico y científico, turismo de naturaleza, turismo de salud, turismo de negocios, viajes de incentivo, turismo de cruceros, turismo de intereses sociales, turismo de intercambio profesional, turismo religioso, turismo de hobbies, aficiones, turismo de bodas y lunas de miel, turismo de aventura y turismo de parques temáticos.' query='Cuales son las distintas formas de turismo' response='Las distintas formas de turismo mencionadas son: turismo de sol y playa, turismo cultural, turismo de reuniones, turismo deportivo, turismo náutico, turismo académico y científico, turismo de naturaleza, turismo de salud, turismo de negocios, viajes de incentivo, turismo de cruceros, turismo de intereses sociales, turismo de intercambio profesional, turismo religioso, turismo 

In [8]:
r1

'Las distintas formas de turismo mencionadas son: turismo de sol y playa, turismo cultural, turismo de reuniones, turismo deportivo, turismo náutico, turismo académico y científico, turismo de naturaleza, turismo de salud, turismo de negocios, viajes de incentivo, turismo de cruceros, turismo de intereses sociales, turismo de intercambio profesional, turismo religioso, turismo de hobbies, aficiones, turismo de bodas y lunas de miel, turismo de aventura y turismo de parques temáticos.'

In [9]:
r2 =bot.reply(query2)

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
r2

'Los pasos principales de la organización y el desarrollo de un viaje incluyen la preparación, ejecución y post-viaje. En la fase de preparación, hay actividades relacionadas con los hoteles, restaurantes y el turismo, administración de proyectos de objetivos múltiples, regulación de sectores comerciales diversos, administración de actividades laborales generales, aplicación de políticas de investigación y desarrollo, ejecución de medidas de política de desarrollo regional y regulación y administración de actividades relacionadas con los transportes y las comunicaciones. En la fase de ejecución, se incluyen las actividades recreativas con fines de esparcimiento al aire libre o bajo techo, con la participación de atletas, así como la explotación de las instalaciones en que se realizan tales actividades. En la fase de post-viaje, se administra la información de que disponen para la toma de decisiones y se reducen costos. Las entidades participantes en el turismo incluyen hoteles, restaur